# Which player has won the most tournaments in a single month?
> #### I didn't find any column in any table that mentioned the month of tournament so I tried to find Player with the most tournament wins

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

home_team_dataframe = pd.DataFrame()
for file in os.listdir("raw/home_team_parquet"):
    single_stats = pd.read_parquet("raw/home_team_parquet/" + file)
    home_team_dataframe = pd.concat([home_team_dataframe, single_stats], axis= 'rows', ignore_index=True)

away_team_dataframe = pd.DataFrame()
for file in os.listdir("raw/away_team_parquet"):
    single_stats = pd.read_parquet("raw/away_team_parquet/" + file)
    away_team_dataframe = pd.concat([away_team_dataframe, single_stats], axis= 'rows', ignore_index=True)

event_dataframe = pd.DataFrame()
for file in os.listdir("raw/event_parquet"):
    single_stats = pd.read_parquet("raw/event_parquet/" + file)
    event_dataframe = pd.concat([event_dataframe, single_stats], axis= 'rows', ignore_index=True)

tournament_dataframe = pd.DataFrame()
for file in os.listdir("raw/tournament_parquet"):
    single_stats = pd.read_parquet("raw/tournament_parquet/" + file)
    tournament_dataframe = pd.concat([tournament_dataframe, single_stats], axis= 'rows', ignore_index=True)
  

In [2]:
# گام 1: ترکیب اطلاعات برنده‌های مسابقات
# فیلتر کردن مسابقات برنده شده توسط تیم خانه (winner_code = 1)
home_wins = event_dataframe[event_dataframe['winner_code'] == 1].merge(home_team_dataframe, on='match_id')

# فیلتر کردن مسابقات برنده شده توسط تیم میهمان (winner_code = 2)
away_wins = event_dataframe[event_dataframe['winner_code'] == 2].merge(away_team_dataframe, on='match_id')

# ادغام لیست برندگان تیم خانه و میهمان
all_wins = pd.concat([home_wins[['match_id', 'player_id']], away_wins[['match_id', 'player_id']]])

# گام 2: ادغام اطلاعات تورنمنت‌ها با برندگان
all_wins = all_wins.merge(tournament_dataframe[['match_id', 'tournament_id', 'tournament_name']], on='match_id')

# گام 3: شمارش تعداد تورنمنت‌هایی که هر بازیکن برنده شده است
tournament_wins = all_wins.groupby('player_id')['tournament_id'].nunique().reset_index(name='tournament_count')

# گام 4: پیدا کردن بازیکنی که بیشترین تعداد تورنمنت‌ها را برنده شده است
top_player = tournament_wins.loc[tournament_wins['tournament_count'].idxmax()]

# استخراج اطلاعات بازیکن
top_player_id = top_player['player_id']
top_player_wins = top_player['tournament_count']

# گام 5: پیدا کردن نام بازیکن از روی player_id
if top_player_id in home_team_dataframe['player_id'].values:
    top_player_name = home_team_dataframe[home_team_dataframe['player_id'] == top_player_id]['name'].values[0]
elif top_player_id in away_team_dataframe['player_id'].values:
    top_player_name = away_team_dataframe[away_team_dataframe['player_id'] == top_player_id]['name'].values[0]
else:
    top_player_name = "No Player Found"

print(f"Player with most tournament wins: {top_player_name} (ID: {top_player_id})")
print(f"Number of tournament wins: {top_player_wins}")

Player with most tournament wins: Kubka M. (ID: 294815)
Number of tournament wins: 2
